# MIMIC-FHIR Tutorial Using HAPI 
This tutorial will walk through using the mimic-fhir resources with a HAPI FHIR server. The main goals will be:
- Import resources into server (delay this for now, just use fully loaded server...)
  - But this is important, how to get a fully loaded server from ndjson, lots of bundles
  - HAPI has not implemented bulk import of ndjson or even $import to the jpa servers
- Export resources from server 
  - Select subset and export
  - $everything to get info for one patient
- $translate to take one of our codes and map to more standard FHIR CodeSystem
- $validate
- $search 
- check data quality, does every patient have a heart rate? Can do this since hapi can handle this

In [ ]:
from pathlib import Path
import requests
import json
import ndjson
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

from fhirclient.models.parameters import Parameters, ParametersParameter
from py_mimic_fhir.lookup import MIMIC_FHIR_RESOURCES

FHIR_SERVER = 'http://localhost:8080/fhir'

## Import into HAPI

In [ ]:
#

## $translate
- Is this worth demoing?

## $everything

In [ ]:
def write_ndjson_from_response(filename, resp, mode = 'w+'):
    with open(filename, mode) as outfile:
        writer = ndjson.writer(outfile)
        for entry in resp.json()['entry']:
            writer.writerow(entry['resource'])

In [ ]:
output_file = 'output/encounter_everything.ndjson'
url = f'{FHIR_SERVER}/Encounter/0eade521-5b02-50df-a02c-54f029bc0ba8/$everything'
resp = requests.get(url=url, headers={"Content-Type": "application/fhir+json"})
write_ndjson_from_response(output_file, resp)

next_url = [link['url'] for link in resp.json()['link'] if link['relation'] == 'next'] 
while next_url:
    url = next_url[0]
    resp = requests.get(url=url, headers={"Content-Type": "application/fhir+json"})
    if resp.json()['resourceType'] == 'Bundle': #Good!
        write_ndjson_from_response(output_file, resp, 'a+')
        next_url = [link['url'] for link in resp.json()['link'] if link['relation'] == 'next'] 
    else:
        print(f"Breaking out due to error: {resp.json()['issue']}")
        break
    

In [ ]:
output_file = 'output/patient_everything.ndjson'
url = f'{FHIR_SERVER}/Patient/b410dd44-7d65-56f9-974f-2751e8aa80e2/$everything?_type=Condition'
resp = requests.get(url=url, headers={"Content-Type": "application/fhir+json"})
write_ndjson_from_response(output_file, resp)

next_url = [link['url'] for link in resp.json()['link'] if link['relation'] == 'next'] 
while next_url:
    url = next_url[0]
    resp = requests.get(url=url, headers={"Content-Type": "application/fhir+json"})
    if resp.json()['resourceType'] == 'Bundle': #Good!
        write_ndjson_from_response(output_file, resp, 'a+')
        next_url = [link['url'] for link in resp.json()['link'] if link['relation'] == 'next'] 
    else:
        print(f"Breaking out due to error: {resp.json()['issue']}")
        break

## $search

In [ ]:
resource='Patient'
url = f'{FHIR_SERVER}/{resource}?gender=male'

resp = requests.get(url, headers={"Content-Type": "application/fhir+json"} )
resp.json()

## Check Data Integrity
- check all patients have heart rate -> observation_chartevents
  - BUT this is way harder to do in a standard fhir server, no reverseResolve

In [ ]:
#